In [1]:
import os

import tensorflow as tf

from datasets import dataset_factory
from deployment import model_deploy
from nets import nets_factory
from preprocessing import preprocessing_factory
from nets import inception_v3_concat as inception_v3
from preprocessing import inception_preprocessing
slim = tf.contrib.slim

In [2]:
dataset = dataset_factory.get_dataset(
        't72willys', 'train', '/root/qq/t72_willys')

    ######################
    # Select the network #
    ######################
network_fn = nets_factory.get_network_fn(
    'inception_v3',
    num_classes=2,
    weight_decay=0.00004,
    is_training=True)
image_preprocessing_fn = preprocessing_factory.get_preprocessing(
        'inception_v3',
        is_training=True)
provider = slim.dataset_data_provider.DatasetDataProvider(
          dataset,
          num_readers=4,
          common_queue_capacity=20 * 32,
          common_queue_min=10 * 32)
[image, label] = provider.get(['image', 'label'])


train_image_size = 299

image = image_preprocessing_fn(image, train_image_size, train_image_size)

images, labels = tf.train.batch(
  [image, label],
  batch_size=32,
  num_threads=4,
  capacity=5 * 32)
labels = slim.one_hot_encoding(
  labels, 2)

In [3]:
def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV3/Logits", "InceptionV3/AuxLogits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]
    
    variables_to_restore = []
    variables_to_retrain = []
    
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
        else:
            variables_to_retrain.append(var)
    
    return slim.assign_from_checkpoint_fn(
      '/root/qq/slim_test/training/inceptionv3/model.ckpt-8779', variables_to_restore, ignore_missing_vars=True)

In [4]:

with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
    logits, end_points = inception_v3.inception_v3(images, num_classes=2, is_training=True)

with tf.variable_scope('custom'):
    pre1 = end_points['PreLogits']
    pre2 = end_points['PreLogits']

    net = tf.concat([pre1, pre2], axis=3, name='asdf')
    logits = slim.conv2d(net, 2, [1,1], activation_fn=None, normalizer_fn=None)

    logits = tf.squeeze(logits, [1,2])

    slim.losses.softmax_cross_entropy(logits, labels)
    total_loss = slim.losses.get_total_loss()

    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    train_op = slim.learning.create_train_op(total_loss, optimizer)


print ("################################")

final_loss = slim.learning.train(
    train_op, 
    logdir='/root/qq/slim_test/training/inceptionv3_concat',
    number_of_steps=1000)
    


Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
################################
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /root/qq/slim_test/training/inceptionv3_concat/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:custom/global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global step 1: loss = 1.3881 (4.220 sec/step)
INFO:tensorflow:global step 2: loss = 1.5170 (0.351 sec/step)
INFO:tensorflow:global step 3: l

INFO:tensorflow:global step 93: loss = 1.3896 (0.411 sec/step)
INFO:tensorflow:global step 94: loss = 1.2735 (0.428 sec/step)
INFO:tensorflow:global step 95: loss = 1.4882 (0.395 sec/step)
INFO:tensorflow:global step 96: loss = 1.3256 (0.407 sec/step)
INFO:tensorflow:global step 97: loss = 1.3879 (0.421 sec/step)
INFO:tensorflow:global step 98: loss = 1.5350 (0.428 sec/step)
INFO:tensorflow:global step 99: loss = 1.2830 (0.407 sec/step)
INFO:tensorflow:global step 100: loss = 1.3784 (0.444 sec/step)
INFO:tensorflow:global step 101: loss = 1.2298 (0.432 sec/step)
INFO:tensorflow:global step 102: loss = 1.3103 (0.408 sec/step)
INFO:tensorflow:global step 103: loss = 1.1763 (0.413 sec/step)
INFO:tensorflow:global step 104: loss = 1.1451 (0.410 sec/step)
INFO:tensorflow:global step 105: loss = 1.3041 (0.402 sec/step)
INFO:tensorflow:global step 106: loss = 1.3902 (0.436 sec/step)
INFO:tensorflow:global step 107: loss = 1.0843 (0.397 sec/step)
INFO:tensorflow:global step 108: loss = 1.2520 

INFO:tensorflow:global step 222: loss = 1.3729 (0.409 sec/step)
INFO:tensorflow:global step 223: loss = 1.3272 (0.400 sec/step)
INFO:tensorflow:global step 224: loss = 1.3636 (0.441 sec/step)
INFO:tensorflow:global step 225: loss = 1.1921 (0.434 sec/step)
INFO:tensorflow:global step 226: loss = 1.1907 (0.417 sec/step)
INFO:tensorflow:global step 227: loss = 1.2100 (0.400 sec/step)
INFO:tensorflow:global step 228: loss = 1.2157 (0.405 sec/step)
INFO:tensorflow:global step 229: loss = 1.4519 (0.390 sec/step)
INFO:tensorflow:global step 230: loss = 1.1315 (0.404 sec/step)
INFO:tensorflow:global step 231: loss = 1.1181 (0.459 sec/step)
INFO:tensorflow:global step 232: loss = 1.1909 (0.416 sec/step)
INFO:tensorflow:global step 233: loss = 1.2840 (0.454 sec/step)
INFO:tensorflow:global step 234: loss = 1.2743 (0.413 sec/step)
INFO:tensorflow:global step 235: loss = 1.3789 (0.438 sec/step)
INFO:tensorflow:global step 236: loss = 1.3170 (0.432 sec/step)
INFO:tensorflow:global step 237: loss = 

INFO:tensorflow:global step 351: loss = 1.1056 (0.410 sec/step)
INFO:tensorflow:global step 352: loss = 1.2415 (0.424 sec/step)
INFO:tensorflow:global step 353: loss = 1.4330 (0.396 sec/step)
INFO:tensorflow:global step 354: loss = 1.3308 (0.420 sec/step)
INFO:tensorflow:global step 355: loss = 1.2670 (0.409 sec/step)
INFO:tensorflow:global step 356: loss = 1.1887 (0.415 sec/step)
INFO:tensorflow:global step 357: loss = 1.2355 (0.415 sec/step)
INFO:tensorflow:global step 358: loss = 1.2295 (0.391 sec/step)
INFO:tensorflow:global step 359: loss = 1.1922 (0.402 sec/step)
INFO:tensorflow:global step 360: loss = 1.2946 (0.427 sec/step)
INFO:tensorflow:global step 361: loss = 1.1664 (0.407 sec/step)
INFO:tensorflow:global step 362: loss = 1.2098 (0.413 sec/step)
INFO:tensorflow:global step 363: loss = 1.1949 (0.432 sec/step)
INFO:tensorflow:global step 364: loss = 1.1246 (0.425 sec/step)
INFO:tensorflow:global step 365: loss = 1.2963 (0.411 sec/step)
INFO:tensorflow:global step 366: loss = 

INFO:tensorflow:global step 480: loss = 1.0230 (0.427 sec/step)
INFO:tensorflow:global step 481: loss = 1.1227 (0.443 sec/step)
INFO:tensorflow:global step 482: loss = 1.2380 (0.397 sec/step)
INFO:tensorflow:global step 483: loss = 1.0280 (0.417 sec/step)
INFO:tensorflow:global step 484: loss = 1.1049 (0.414 sec/step)
INFO:tensorflow:global step 485: loss = 1.0776 (0.404 sec/step)
INFO:tensorflow:global step 486: loss = 1.2331 (0.409 sec/step)
INFO:tensorflow:global step 487: loss = 1.1376 (0.426 sec/step)
INFO:tensorflow:global step 488: loss = 1.4055 (0.406 sec/step)
INFO:tensorflow:global step 489: loss = 1.5093 (0.426 sec/step)
INFO:tensorflow:global step 490: loss = 1.3866 (0.420 sec/step)
INFO:tensorflow:global step 491: loss = 1.3028 (0.421 sec/step)
INFO:tensorflow:global step 492: loss = 1.0530 (0.389 sec/step)
INFO:tensorflow:global step 493: loss = 1.1675 (0.412 sec/step)
INFO:tensorflow:global step 494: loss = 1.2126 (0.398 sec/step)
INFO:tensorflow:global step 495: loss = 

INFO:tensorflow:global step 609: loss = 1.1820 (0.397 sec/step)
INFO:tensorflow:global step 610: loss = 0.9644 (0.420 sec/step)
INFO:tensorflow:global step 611: loss = 1.0194 (0.402 sec/step)
INFO:tensorflow:global step 612: loss = 1.0977 (0.412 sec/step)
INFO:tensorflow:global step 613: loss = 0.9436 (0.405 sec/step)
INFO:tensorflow:global step 614: loss = 1.0096 (0.430 sec/step)
INFO:tensorflow:global step 615: loss = 1.0650 (0.436 sec/step)
INFO:tensorflow:global step 616: loss = 1.0222 (0.430 sec/step)
INFO:tensorflow:global step 617: loss = 1.4363 (0.420 sec/step)
INFO:tensorflow:global step 618: loss = 1.2870 (0.399 sec/step)
INFO:tensorflow:global step 619: loss = 1.1181 (0.407 sec/step)
INFO:tensorflow:global step 620: loss = 1.2257 (0.422 sec/step)
INFO:tensorflow:global step 621: loss = 0.9303 (0.425 sec/step)
INFO:tensorflow:global step 622: loss = 1.1572 (0.413 sec/step)
INFO:tensorflow:global step 623: loss = 1.1169 (0.410 sec/step)
INFO:tensorflow:global step 624: loss = 

INFO:tensorflow:global step 738: loss = 0.9708 (0.439 sec/step)
INFO:tensorflow:global step 739: loss = 0.8856 (0.446 sec/step)
INFO:tensorflow:global step 740: loss = 1.2024 (0.464 sec/step)
INFO:tensorflow:global step 741: loss = 1.0821 (0.449 sec/step)
INFO:tensorflow:global step 742: loss = 1.0100 (0.485 sec/step)
INFO:tensorflow:global step 743: loss = 0.9213 (0.447 sec/step)
INFO:tensorflow:global step 744: loss = 1.0134 (0.428 sec/step)
INFO:tensorflow:global step 745: loss = 0.9979 (0.414 sec/step)
INFO:tensorflow:global step 746: loss = 1.2870 (0.423 sec/step)
INFO:tensorflow:global step 747: loss = 1.2104 (0.412 sec/step)
INFO:tensorflow:global step 748: loss = 1.2324 (0.396 sec/step)
INFO:tensorflow:global step 749: loss = 0.9809 (0.417 sec/step)
INFO:tensorflow:global step 750: loss = 0.9758 (0.411 sec/step)
INFO:tensorflow:global step 751: loss = 1.0612 (0.436 sec/step)
INFO:tensorflow:global step 752: loss = 1.0988 (0.436 sec/step)
INFO:tensorflow:global step 753: loss = 

INFO:tensorflow:global step 867: loss = 1.0633 (0.411 sec/step)
INFO:tensorflow:global step 868: loss = 1.0703 (0.400 sec/step)
INFO:tensorflow:global step 869: loss = 0.9547 (0.417 sec/step)
INFO:tensorflow:global step 870: loss = 1.1474 (0.413 sec/step)
INFO:tensorflow:global step 871: loss = 0.8960 (0.403 sec/step)
INFO:tensorflow:global step 872: loss = 1.0317 (0.412 sec/step)
INFO:tensorflow:global step 873: loss = 1.4121 (0.412 sec/step)
INFO:tensorflow:global step 874: loss = 1.1614 (0.411 sec/step)
INFO:tensorflow:global step 875: loss = 1.1758 (0.422 sec/step)
INFO:tensorflow:global step 876: loss = 1.1091 (0.405 sec/step)
INFO:tensorflow:global step 877: loss = 1.1681 (0.430 sec/step)
INFO:tensorflow:global step 878: loss = 0.9828 (0.424 sec/step)
INFO:tensorflow:global step 879: loss = 1.1279 (0.404 sec/step)
INFO:tensorflow:global step 880: loss = 1.0774 (0.410 sec/step)
INFO:tensorflow:global step 881: loss = 1.0477 (0.418 sec/step)
INFO:tensorflow:global step 882: loss = 

INFO:tensorflow:global step 996: loss = 1.0117 (0.402 sec/step)
INFO:tensorflow:global step 997: loss = 1.0496 (0.424 sec/step)
INFO:tensorflow:global step 998: loss = 0.8818 (0.424 sec/step)
INFO:tensorflow:global step 999: loss = 1.0392 (0.408 sec/step)
INFO:tensorflow:global step 1000: loss = 1.2181 (0.394 sec/step)
INFO:tensorflow:Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.
